In [ ]:
from wordcloud import WordCloud 
import matplotlib.pyplot as plt

def bar(orientation,data,title,color,labels):
    plt.figure(figsize = (15, 10))
    if orientation == 'horizontal': 
        plt.barh(data[0],data[1],color=color)
    else: 
        plt.bar(data[0],data[1],color=color)
    plt.title(title, fontsize=20)
    plt.xlabel(labels[0], fontsize=15)
    plt.xticks(fontsize=12)
    plt.ylabel(labels[1], fontsize=15)
    plt.yticks(fontsize=10)
    plt.show()


def wordcloud(counts, title): 
    word_cloud = WordCloud(background_color = "white", width=3000, height=2000, max_words=500, collocations=True).generate_from_frequencies(counts)
    plt.figure(figsize=(20,10))
    plt.imshow(word_cloud)
    plt.axis("off")
    plt.title(title,fontsize=18)
    plt.show()

In [1]:
from bibleMarginalia import *
from getTexts import * 
# EP: /Users/amycweng/Digital Humanities/eebotcp/texts
# TCP: /Users/amycweng/Digital Humanities/TCP 

csv_data = pd.read_csv('/Users/amycweng/Digital Humanities/sermons.csv')
tcpIDs = [ _ for _ in csv_data['id']]

all_specials = []
for idx, tcpID in enumerate(tcpIDs): 
    path = findTextTCP(tcpID)
    notes, specials = getMarginalia(path)
    for case in specials: 
        all_specials.append(case.split(' ')[0])
    with open(f'/Users/amycweng/Digital Humanities/charityMargin/{tcpID}NOTES.txt','w') as file: 
        for note in notes[0]: 
            file.write(f'{note}\n')
    if len(notes[1]) > 0: 
        print(f'Outlier formatting for {tcpID}: ', notes[1])
    if (idx+1) % 10 == 0 and idx != 0: 
        print(f'Processed {idx+1} files.')
print('Possibly missing abbreviations: ', Counter(all_specials))

Processed 10 files.
Processed 20 files.
Processed 30 files.
Processed 40 files.
Processed 50 files.
Processed 60 files.
Processed 70 files.
Possibly missing abbreviations:  Counter({'apoc': 15, 'c': 15, 'l': 10, 'cap': 9, 'fur': 8, 'chap': 7, 'io': 7, 'sirac': 7, 'ibid': 6, 'trist': 5, 'ma': 4, 'wisd': 4, 'amor': 4, 'sal': 4, 'trinum': 4, 'heaut': 4, 'vers': 4, 'v': 4, 'thyest': 4, 'amph': 4, 'med': 4, 'q': 3, 'a': 3, 'apo': 3, 'sai': 3, 'capt': 3, 'tom': 3, 'stich': 3, 'pont': 3, 'thy': 3, 'exerc': 3, 'agam': 3, 'sa': 3, 'part': 3, 'octav': 3, 'adelph': 3, 'cum': 3, 'esal': 2, 'pseud': 2, 'pers': 2, 'm': 2, 'apc': 2, 'n': 2, 'in': 2, 'dut': 2, 'irem': 2, 'trucul': 2, 't': 2, 'var': 2, 'ter': 2, 'ponto': 2, 'rud': 2, 'mostell': 2, 'eun': 2, 'tibull': 2, 'controv': 2, 'point': 2, 'hippol': 2, 'hosca': 2, 'micha': 2, 'paris': 2, 'art': 2, 'timo': 2, 'troad': 2, 'esd': 2, 'truc': 2, 'thren': 2, 'merc': 2, 'curcul': 2, 'serm': 2, 'reg': 2, 'car': 2, 'lib': 2, 'verse': 2, 'er': 2, 'paral': 

In [ ]:
from charity_citations import * 

all_notes = []
all_books = []
all_chapters = []
charity_margin = {}
all_margin = {}

folder = '/Users/amycweng/Digital Humanities/charityMargin'
for notesfile in os.listdir(folder): 
    tcpID = notesfile.split('NOTES')[0]
    charity_margin[tcpID] = []

    notesfile = open(f'{folder}/{notesfile}','r')
    notes = [note.strip('\n') for note in notesfile.readlines()]
    notesfile.close()

    all_notes.extend(notes)
    all_margin[tcpID] = notes
    for n in notes:             
        if n in bible_charity: 
            charity_margin[tcpID].append(n)
        book = n.split(' ')
        if book[0] == '1' or book[0] == '2': 
            all_books.append(f'{book[0]} {book[1]}')
            chapter = book[2].split(':')[0]
            all_chapters.append(f'{book[0]} {book[1]} {chapter}')
        else: 
            all_books.append(book[0])
            chapter = book[1].split(':')[0]
            all_chapters.append(f'{book[0]} {chapter}')

In [ ]:
charity_citations_file = open(f'marginalia.charity.sermons.txt','w')
all_citations_file = open(f'marginalia.all.sermons.txt','w')
for tcpID, margin in charity_margin.items(): 
    charity_citations_file.write(f'{tcpID} -- {"; ".join(margin)}\n')
    all_citations_file.write(f'{tcpID} -- {"; ".join(all_margin[tcpID])}\n')
charity_citations_file.close()
all_citations_file.close()

In [ ]:
book_counts = Counter(all_books).most_common(n=25)
print(book_counts)
x,y = [],[]
for word,freq in book_counts: 
    x.append(word)
    y.append(freq)
bar('horizontal',(x,y),'Most Commonly Cited Biblical or Apocryphal Books in the Marginalia of the Charity Sermons Dataset','darkblue',('Book','Frequency'))

In [ ]:
chapter_counts = Counter(all_chapters)
print(chapter_counts.most_common(n=25))
wordcloud(chapter_counts,'Biblical or Apocryphal Chapters in the Marginalia of the Charity Sermons Dataset')

In [ ]:
passage_counts = Counter(all_notes)
print(passage_counts.most_common(n=25))
wordcloud(passage_counts,'Biblical or Apocryphal Passages in the Marginalia of the Charity Sermons Dataset')

In [ ]:
newTestament = ['Matthew', 'Mark', 'Luke', 'John','Acts','Romans','1 Corinthians','2 Corinthians','Galatians','Ephesians','Philippians','Colossians','1 Thessalonians','2 Thessalonians','1 Timothy','2 Timothy','Titus','Philemon','Hebrews','James','1 Peter','2 Peter','1 John', '2 John','3 John','Jude','Revelation']
oldTestament = ['Genesis','Exodus','Leviticus','Numbers','Deuteronomy','Joshua','Judges','Ruth','1 Samuel','2 Samuel','1 Kings','2 Kings','1 Chronicles','2 Chronicles','Ezra','Nehemiah','Esther','Job','Psalms','Proverbs','Ecclesiastes','Canticles','Isaiah','Jeremiah','Lamentations','Ezekiel','Daniel','Hosea','Joel','Amos','Obadiah','Jonah','Micah','Nahum','Habakkuk','Zephaniah','Haggai','Zechariah','Malachi']

newTcount, oldTcount = 0,0

book_counts = dict(Counter(all_books))
for book, count in book_counts.items(): 
    if book in newTestament: 
        newTcount += count 
    elif book in oldTestament: 
        oldTcount += count
print(f'New T {newTcount} and Old T {oldTcount}')

In [ ]:
all_charity = []
for c_list in charity_margin.values():
    all_charity.extend(c_list)
print(f'There are {len(all_charity)} citations of charity-related passages.')
charity_counts = dict(Counter(all_charity))
print('Lines cited only once: ', [line for line, freq in charity_counts.items() if freq == 1])
charity_counts = sorted([(line, freq) for line, freq in charity_counts.items() if freq > 1],key=lambda k: (k[1], k[0]))
x,y = [],[]
for word,freq in charity_counts: 
    x.append(word)
    y.append(freq)
bar('horizontal',(x,y),'Biblical Passages on Charity in the Marginalia of the Charity Sermons Dataset','palevioletred',('Frequency','Passage'))